#### As of September 2025, we are gradually understanding the limitation of the current FBA model in simulating knockdown behavior of kinetic genes and many other, including sparse solution matrix (flow partition). Flow paritition is not the focus of this script; instead, we are focusing on the intricate relationship between the rest of the objective weights: homeostatic, secretion, efficiency, and kinetic, to find a balance of the weights. The reason why we are analyzing this is because we have noticed, through Riley's new gene metabolism runs, that modifying the kinetic target of all reaction have no effect on the overall fluxome/production of homeostatic metabolite, which is not physiologial. This is might be because the homeostatic weight is so much higher than the kinetic weight such that we would rather take penalty in the kinetic term of the objective than to sacrifice not making enough homeostatic metabolite to sustain growth. The goal of this script is to find out the balance between the objective weights and choose them appropriatly so that the basal sim doesn't change but if we were to simulate pathway knockdowns, differences will be more prominent.

In [1]:
import pandas as pd
import os, dill
import numpy as np
import cvxpy as cp
from typing import Iterable, Optional, Mapping, cast
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from ecoli.processes.metabolism_redux_classic import NetworkFlowModel, FlowResult
os.chdir(os.path.expanduser('~/dev/vEcoli'))

%load_ext autoreload

In [2]:
# import sim
time = '600'
date = '2025-09-30'
experiment = 'output_objective_weights'
condition = 'basal'
entry = f'{experiment}_{time}_{date}'
folder = f'out/objective_weight/{condition}/{entry}/'

output = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
output = output['agents']['0']
fba = output['listeners']['fba_results']
bulk = pd.DataFrame(output['bulk'])
f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

In [3]:
# get commonly stored variables
metabolism = agent['ecoli-metabolism-redux-classic']
stoichiometry = metabolism.stoichiometry.copy()
reaction_names = metabolism.reaction_names
fba_new_reaction_ids = metabolism.parameters["fba_new_reaction_ids"]
fba_reaction_ids_to_base_reaction_ids = metabolism.parameters['fba_reaction_ids_to_base_reaction_ids']
metabolites = metabolism.metabolite_names.copy()
binary_kinetic_idx = metabolism.binary_kinetic_idx
exchange_molecules = metabolism.exchange_molecules
kinetic_enzymes = metabolism.parameters['kinetic_constraint_enzymes']

S = stoichiometry .copy()
S = pd.DataFrame(S, index=metabolites , columns=reaction_names )
homeostatic_count = pd.DataFrame(fba["homeostatic_metabolite_counts"], columns=metabolism.homeostatic_metabolites).mean(axis=0)
homeostatic = pd.DataFrame(fba["target_homeostatic_dmdt"], columns=metabolism.homeostatic_metabolites).mean(axis=0)
maintenance = pd.DataFrame(fba["maintenance_target"][1:], columns=['maintenance_reaction']).mean(axis=0)
kinetic = pd.DataFrame(fba["target_kinetic_fluxes"], columns=metabolism.kinetic_constraint_reactions).mean(axis=0).copy()

kinetic_reaction_ids = metabolism.kinetic_constraint_reactions
allowed_exchange_uptake = metabolism.allowed_exchange_uptake
FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474"]

In [4]:
# get objective terms across time
counts_to_molar = metabolism.counts_to_molar.asNumber()

homeostatic_term = np.asarray(fba['homeostatic_term'])/counts_to_molar #change to concentration
secretion_term = np.asarray(fba['secretion_term'])/counts_to_molar
efficiency_term = np.asarray(fba['efficiency_term'])/counts_to_molar
kinetic_term = np.asarray(fba['kinetic_term'])/counts_to_molar

time = np.arange(len(homeostatic_term))

objective_weights_sim = {
            "secretion": 0.01,
            "efficiency": 0.000001,
            "kinetics": 0.00001,
            "homeostatic": 1,
}

# Define helper functions

In [50]:
def get_subset_S(S, met_of_interest):
    S_met = S.loc[met_of_interest, :]
    S_met = S_met.loc[:,~np.all(S_met == 0, axis=0)]
    return S_met, S_met.columns

def get_keys(dict, value):
    return [key for key in dict if np.any(np.isin(value, dict[key]))]

def test_NetworkFlowModel(objective_weights,
                          uptake_addition = set([]), uptake_removal = set([]), new_exchange_molecules = set([]),
                          add_metabolite = None, add_reaction = None, add_kinetic = None, remove_reaction = None, force_reaction = None,
                          add_homeostatic_demand = None, solver_choice=cp.GLOP):
    # update exchanges
    uptake = metabolism.allowed_exchange_uptake.copy()
    uptake = set(uptake)
    uptake = uptake | uptake_addition
    uptake = uptake - uptake_removal

    exchange_molecules = metabolism.exchange_molecules.copy()
    exchange_molecules = exchange_molecules | new_exchange_molecules

    # update stoichiometry
    reaction_names = metabolism.reaction_names.copy()
    kinetic_reaction_ids = metabolism.kinetic_constraint_reactions.copy()
    kinetic = pd.DataFrame(fba["target_kinetic_fluxes"], columns=metabolism.kinetic_constraint_reactions).loc[24, :].copy()
    metabolites = metabolism.metabolite_names.copy()
    homeostatic_counts = homeostatic_count.copy() * metabolism.counts_to_molar.asNumber()

    S_new = stoichiometry.copy()

    if add_metabolite is not None: # add to metabolites list because they are currently not included in the model
        for m in add_metabolite:
            if m not in metabolites:
                metabolites.append(m)
        # append rows of zeros to S_new of length add_metabolite
        S_new = np.concatenate((S_new, np.zeros((len(add_metabolite), S_new.shape[1]))), axis=0)

    if add_reaction is not None:
        # assert add_reaction is a dictionary
        assert isinstance(add_reaction, dict)

        for r,s in add_reaction.items():
            if r not in reaction_names:
                reaction_names.append(r)
            # append columns of reaction stoich to S_new of length add_reaction
            new_reaction = np.zeros((S_new.shape[0], 1))
            for m, v in s.items():
                new_reaction[metabolites.index(m), 0] = v
            S_new = np.concatenate((S_new, new_reaction), axis=1)

    if add_kinetic is not None:
        # assert add_kinetic is a dictionary
        assert isinstance(add_kinetic, dict)

        for r, v in add_kinetic.items():
            if r not in kinetic_reaction_ids:
                kinetic_reaction_ids.append(r)
                kinetic[r] = v
            if r in kinetic_reaction_ids:
                kinetic[r] = v

    if remove_reaction is not None:
        for r in remove_reaction:
            r_idx = reaction_names.index(r)
            S_new = np.delete(S_new, r_idx, axis=1)
            reaction_names.remove(r)
            if r in kinetic_reaction_ids:
                kinetic_reaction_ids.remove(r)
                del kinetic[r]

    if force_reaction is not None:
        force_reaction_idx = np.array([reaction_names.index(r) for r in force_reaction])
    else:
        force_reaction_idx = force_reaction

    if add_homeostatic_demand is not None:
        # assert add_homeostatic_demand is a set
        assert isinstance(add_homeostatic_demand, list)

        for met in add_homeostatic_demand:
            homeostatic[met] = 100
            homeostatic_counts[met] = 1

    # Solve NetworkFlowModel
    model = NetworkFlowModel(
            stoich_arr=S_new,
            metabolites=metabolites,
            reactions=reaction_names,
            homeostatic_metabolites=metabolism.homeostatic_metabolites,
            kinetic_reactions=kinetic_reaction_ids,
            free_reactions=FREE_RXNS)
    model.set_up_exchanges(exchanges=exchange_molecules, uptakes=uptake)
    solution: FlowResult = model.solve(
            homeostatic_concs=homeostatic_counts, # in conc
            homeostatic_dm_targets=np.array(list(dict(homeostatic).values())), # *10^7
            maintenance_target=maintenance, # *10^6 ish
            kinetic_targets=np.array(list(dict(kinetic).values())), # *10^6 ish
            # binary_kinetic_idx=binary_kinetic_idx, #7646
            binary_kinetic_idx=None,
            force_flow_idx=force_reaction_idx,
            objective_weights=objective_weights, #same
            upper_flux_bound= 1000000000, # increase to 10^9 because notebook runs FlowResult using Counts, WC runs using conc.
            solver=solver_choice) #SCS. ECOS, MOSEK
    return solution.objective, solution.velocities, reaction_names, S_new, solution.kinetic_term, solution.homeostatic_term, solution.secretion_term, solution.efficiency_term, solution.exchanges

# Plot 1: Visualize how well each objective term is being met across the sim

In [6]:
from plotly.colors import qualitative as q

spec_term = {
    "Homeostatic": (homeostatic_term, objective_weights_sim['homeostatic']),
    "Secretion":   (secretion_term,   objective_weights_sim['secretion']),
    "Efficiency":  (efficiency_term,  objective_weights_sim['efficiency']),
    "Kinetic":     (kinetic_term,     objective_weights_sim['kinetics']),
}

spec_no_weights = {
    "Homeostatic": (homeostatic_term/objective_weights_sim['homeostatic'], objective_weights_sim['homeostatic']),
    "Secretion":   (secretion_term/objective_weights_sim['secretion'],     objective_weights_sim['secretion']),
    "Efficiency":  (efficiency_term/objective_weights_sim['efficiency'],   objective_weights_sim['efficiency']),
    "Kinetic":     (kinetic_term/objective_weights_sim['kinetics'],        objective_weights_sim['kinetics']),
}


palette = q.Plotly  # consistent colors across both sets
colors = {name: palette[i % len(palette)] for i, name in enumerate(spec_term)}


fig = go.Figure()

# Add weighted (solid) + unweighted (dotted) per term
for i, (name, (y_w, w)) in enumerate(spec_term.items()):
    # solid, weighted
    fig.add_trace(go.Scatter(
        x=time, y=y_w, mode="lines",
        name=f"{name} (weighted)",
        line=dict(color=colors[name], dash="solid"),
        legendgroup=name,
        legendgrouptitle_text=name if i == 0 else None,
        hovertemplate=f"{name.lower()} weight={w}<br>{name.lower()} term value=%{{y:.4g}}<extra></extra>"
    ))
    # dotted, unweighted
    y_uw, w_uw = spec_no_weights[name]
    fig.add_trace(go.Scatter(
        x=time, y=y_uw, mode="lines",
        name=f"{name} (unweighted)",
        line=dict(color=colors[name], dash="dot"),
        legendgroup=name,
        hovertemplate=f"{name.lower()} (no weight)<br>{name.lower()} value=%{{y:.4g}}<extra></extra>"
    ))

fig.update_layout(
    title="Change of Objective Term Values Over Sim",
    xaxis_title="Time (s)",
    yaxis=dict(title="Objective Term Value", type="log", tickformat=".0e"),
    template="plotly_white",
    legend=dict(title="Terms", groupclick="togglegroup"),  # click once to toggle both lines in a group
    width=1200, height=600,
)

fig.show()
fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/objective_weights/change of objective term values over a sim.png", scale=3)

### Comment: The homeostatic term in the objective has a weird plateu where the rest seems pretty constant throughout the sim. Perhaps, homeostatic demand can't be achieved fully because it's capped by other constraints, so there is always more demand the next time step that can be met, therefore the gradual inscrease of unmet need

# Plot 2: Sensitivity of each objective term in response to ranges of objective weight of other terms

In [51]:
# -----------------------------
# Utilities
# -----------------------------
OBJ_KEYS = ["homeostatic", "secretion", "efficiency", "kinetics"]
LABEL = {k: k.capitalize() if k != "kinetics" else "Kinetic" for k in OBJ_KEYS}
COLORS = {LABEL[k]: c for k, c in zip(OBJ_KEYS, px.colors.qualitative.Plotly)}

def _floor_for_log(arr, floor=1e-6):
    """Avoid zeros on log axes."""
    a = np.asarray(arr, dtype=float)
    a[a <= 0] = floor
    return a

def _extract_terms(call_result):
    """
    Unpack your model's return into a dict of raw, unweighted terms.
    Expected: oofv, _, _, _, kinetic, homeostatic, secretion, efficiency
    """
    oofv, _, _, _, kinetic, homeo, secr, eff, _ = call_result
    return {
        "homeostatic": float(homeo),
        "secretion":   float(secr),
        "efficiency":  float(eff),
        "kinetics":    float(kinetic),
    }

def sweep_one_weight(
    swept_key: str,
    sweep_vals: np.ndarray,
    base_weights: dict,
    model_callable,
    catch_errors: bool = False,
):
    """
    Run the model while sweeping a single weight (swept_key).
    Returns:
        x_used:      np.array of weights actually used
        raw:         dict of lists for raw terms (unweighted)
        weighted:    dict of lists for weighted terms (per-sweep correct)
    """
    raw = {k: [] for k in OBJ_KEYS}
    weighted = {k: [] for k in OBJ_KEYS}
    x_used = []

    for w in sweep_vals:
        ow = dict(base_weights)   # copy
        ow[swept_key] = float(w)

        try:
            terms = _extract_terms(model_callable(ow))
        except Exception as e:
            if catch_errors:
                continue
            else:
                raise

        x_used.append(w)
        for k in OBJ_KEYS:
            raw[k].append(terms[k])
            # weight applied PER TERM at THIS sweep
            curr_weight = ow[k]
            weighted[k].append(terms[k] * curr_weight)

    # convert to np arrays
    x_used = np.array(x_used, dtype=float)
    for d in (raw, weighted):
        for k in d:
            d[k] = np.array(d[k], dtype=float)
    return x_used, raw, weighted


# -----------------------------
# Build the full 2×2 figure
# -----------------------------
# Example sweep ranges (use your exact choices)
sweep_H = np.logspace(-6, 0, 30)
sweep_S = np.logspace(-6, 0, 30)
sweep_E = np.logspace(-9, -3, 30)
sweep_K = np.logspace(-6, 0, 30)

# Copy your starting weights
base = objective_weights_sim.copy()

# Your callable (must match your signature)
def run_model(weights):
    return test_NetworkFlowModel(weights)

In [ ]:
def add_panel(fig, row, col, *,
              swept_key: str,
              sweep_vals: np.ndarray,
              base_weights: dict,
              model_callable,
              panel_title: str,
              catch_errors: bool = False):
    """
    Adds a panel: weighted curves for all terms + (dotted) unweighted for the swept term.
    """
    x, raw, wt = sweep_one_weight(
        swept_key, sweep_vals, base_weights, model_callable, catch_errors=catch_errors
    )

    # Plot weighted curves for all terms
    for k in OBJ_KEYS:
        name = LABEL[k]
        fig.add_trace(
            go.Scatter(
                x=x, y=_floor_for_log(wt[k]),
                mode="lines",
                name=f"{name} (weighted)",
                legendgroup=name,
                line=dict(color=COLORS[name], dash="solid"),
                hovertemplate=f"swept {LABEL[swept_key]} weight=%{{x:.2e}}<br>{name} (weighted)=%{{y:.4g}}<extra></extra>",
            ),
            row=row, col=col
        )

    if row != 1 or col != 1:
        # Hide legend on all but first panel

        fig.update_traces(showlegend=False, row=row, col=col)

    # Add unweighted curve ONLY for the swept term
    name = LABEL[swept_key]
    fig.add_trace(
        go.Scatter(
            x=x, y=_floor_for_log(raw[swept_key]),
            mode="lines",
            name=f"{name} (unweighted)",
            legendgroup=name,
            line=dict(color=COLORS[name], dash="dot"),
            hovertemplate=f"swept {name} weight=%{{x:.2e}}<br>{name} (unweighted)=%{{y:.4g}}<extra></extra>",
        ),
        row=row, col=col
    )

    # Axes cosmetics per panel
    fig.update_xaxes(type="log", tickformat=".0e", title_text=f"Swept {name} weight", row=row, col=col)
    fig.update_yaxes(type="log", tickformat=".0e", title_text="Objective term value", row=row, col=col)
    fig.add_annotation(
        text=panel_title,
        xref=f"x domain",
        yref=f"y domain",
        row=row, col=col,
        x=-0.15, y=1.1, showarrow=False, font=dict(size=16, color="#444")
    )

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Sweep Homeostatic", "Sweep Secretion",
                    "Sweep Efficiency",  "Sweep Kinetics"),
    horizontal_spacing=0.1, vertical_spacing=0.15
)

# A: sweep homeostatic
add_panel(fig, 1, 1,
          swept_key="homeostatic",
          sweep_vals=sweep_H,
          base_weights=base,
          model_callable=run_model,
          panel_title="A")

# B: sweep secretion
add_panel(fig, 1, 2,
          swept_key="secretion",
          sweep_vals=sweep_S,
          base_weights=base,
          model_callable=run_model,
          panel_title="B")

# C: sweep efficiency (with error-catching like your try/except)
add_panel(fig, 2, 1,
          swept_key="efficiency",
          sweep_vals=sweep_E,
          base_weights=base,
          model_callable=run_model,
          panel_title="C",
          catch_errors=True)

# D: sweep kinetics
add_panel(fig, 2, 2,
          swept_key="kinetics",
          sweep_vals=sweep_K,
          base_weights=base,
          model_callable=run_model,
          panel_title="D")

fig.update_layout(
    title="Change of Objective Term Values Across Weight Sweeps",
    template="plotly_white",
    legend=dict(title="Objective terms", groupclick="togglegroup"),
    width=1200, height=800,
    margin=dict(l=60, r=20, t=70, b=40)
)

fig.show()

In [ ]:
fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/objective_weights/sensitivity of objective terms to individual weight changes.png", scale=3, width=1200, height=800,)

# Plot 3: Latin Hypercube Style Sampling of the full 4D weight space to see correlations between weights and objective terms. Heatmap

In [ ]:
from itertools import product
from scipy.stats import spearmanr

def extract_terms(call_result):
    oofv, _, _, _, kinetic, homeo, secr, eff = call_result
    return {
        "homeostatic": float(homeo),
        "secretion":   float(secr),
        "efficiency":  float(eff),
        "kinetics":    float(kinetic),
    }

def sample_weight_space(base, grids):
    """
    grids: dict key->array of values to sample (e.g., np.logspace(...) per weight).
    Returns arrays of samples and a list of dicts of outputs (terms).
    """
    keys = list(grids.keys())
    combos = list(product(*[grids[k] for k in keys]))
    W = []
    outs = []
    for combo in combos:
        ow = dict(base)
        for k, v in zip(keys, combo):
            ow[k] = float(v)
        try:
            terms = extract_terms(run_model(ow))
        except Exception:
            print("fail")
            continue
        W.append({k: float(v) for k, v in zip(keys, combo)})
        outs.append(terms)
    return keys, W, outs

def corr_heatmap(weights_list, outputs_list, weight_keys):
    # Build matrices: X=log10 weights, Y=objective terms
    X = np.array([[np.log10(w[k]) for k in weight_keys] for w in weights_list], float)
    Y = np.array([[o[k] for k in OBJ_KEYS] for o in outputs_list], float)
    # Spearman correlations (rank-based)
    R = np.zeros((len(weight_keys), len(OBJ_KEYS)))
    P = np.zeros_like(R)
    for i in range(R.shape[0]):
        for j in range(R.shape[1]):
            r, p = spearmanr(X[:, i], Y[:, j], nan_policy="omit")
            R[i, j] = r
            P[i, j] = p

    fig = go.Figure(
        data=go.Heatmap(
            z=R, x=[LABEL[k] for k in OBJ_KEYS], y=[LABEL[k] for k in weight_keys],
            zmin=-1, zmax=1, colorscale="RdBu", reversescale=True,
            colorbar=dict(title="Spearman ρ")
        )
    )
    fig.update_layout(
        title="Correlation between log10(weights) and RAW objective terms",
        xaxis_title="Unweighted Objective term",
        yaxis_title="Weight (log10)",
        template="plotly_white",
        width=700, height=400, margin=dict(l=80,r=20,t=60,b=40)
    )
    fig.show()
    return R, P, fig

# ---- Sample across all 4 weights (coarse grid to keep runs reasonable) ----
grids = {
    "homeostatic": np.logspace(-6, 0, 5),
    "secretion":   np.logspace(-6, 0, 5),
    "efficiency":  np.logspace(-9, -3, 5),
    "kinetics":    np.logspace(-6, 0, 5),
}
keys, W_list, O_list = sample_weight_space(base, grids)
R, P, fig_corr = corr_heatmap(W_list, O_list, keys)
# fig_corr.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/objective_weights/objective_weights and term correlation.png", scale = 3)

fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail


# Plot 4: Sweep one weight at a time, and plot correlation matrix of weights vs terms

In [37]:
corr_matrix = np.zeros((len(OBJ_KEYS), len(OBJ_KEYS)))

for swept_key, sweep_vals in zip(OBJ_KEYS, [sweep_H, sweep_S, sweep_E, sweep_K]):
    print(f"Sweeping {swept_key}...")
    if swept_key == 'efficiency':
        # catch errors like before
        x, raw, wt = sweep_one_weight(
            swept_key, sweep_vals, base, run_model, catch_errors=True
        )
    else:
        x, raw, wt = sweep_one_weight(
            swept_key, sweep_vals, base, run_model
        )
    df_raw = pd.DataFrame(raw)
    df_raw['swept_weight'] = np.log10(x)
    corr_row = df_raw.corrwith(df_raw['swept_weight'], method='spearman')
    corr_matrix[OBJ_KEYS.index(swept_key), :] = corr_row.values[:-1] # each row is a swept objective

df_corr_matrix = pd.DataFrame(corr_matrix, index=[LABEL[k] for k in OBJ_KEYS], columns=[LABEL[k] for k in OBJ_KEYS])


Sweeping homeostatic...
Sweeping secretion...
Sweeping efficiency...
Sweeping kinetics...


In [48]:
# create heatmap of df_corr_matrix
fig = go.Figure(
    data=go.Heatmap(
        z=df_corr_matrix.values,
        x=df_corr_matrix.columns,
        y=df_corr_matrix.index,
        zmin=-1, zmax=1,
        colorscale="RdBu",
        reversescale=True,
        colorbar=dict(title="Spearman ρ")
    ))

fig.update_layout(
        title="Correlation between swept weight and unweighted terms",
        xaxis_title="Unweighted Objective term",
        yaxis_title="Weight (log10)",
        template="plotly_white",
        width=700, height=400, margin=dict(l=80,r=20,t=60,b=40)
    )
fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/objective_weights/objective_weights (swepted) and term correlation.png", scale = 3)

In [46]:
# normalize correlation matrix
from sklearn.preprocessing import normalize
t = normalize(df_corr_matrix, axis=1, norm='l1')
df_t = pd.DataFrame(t, index=df_corr_matrix.index, columns=df_corr_matrix.columns)
df_t

,Homeostatic,Secretion,Efficiency,Kinetic
Homeostatic,-0.394327,0.370935,-0.017968,0.216769
Secretion,0.268083,-0.287965,-0.264304,-0.179647
Efficiency,0.251218,-0.023838,-0.382667,0.342277
Kinetic,0.224382,0.241794,0.256405,-0.277419


In [49]:
# plot normalized correlation matrix
fig = go.Figure(
    data=go.Heatmap(
        z=df_t.values,
        x=df_t.columns,
        y=df_t.index,
        zmin=-1, zmax=1,
        colorscale="RdBu",
        reversescale=True,
        colorbar=dict(title="Spearman ρ")
    ))

fig.update_layout(
        title="Correlation between swept weight and unweighted terms",
        xaxis_title="Unweighted Objective term",
        yaxis_title="Weight (log10)",
        template="plotly_white",
        width=700, height=400, margin=dict(l=80,r=20,t=60,b=40)
    )
fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/objective_weights/objective_weights (swept normalized) and term correlation.png", scale = 3)

# Plot 5: Heatmap showing a sweep of homeostatic and secretion weights and the resulting objective term values

In [56]:
grids = {
    "homeostatic": np.logspace(-6, 0, 5),
    "secretion":   np.logspace(-6, 0, 5)
}
keys, W_list, O_list = sample_weight_space(base, grids)

list(product(*[grids[k] for k in keys]))

[(np.float64(1e-06), np.float64(1e-06)),
 (np.float64(1e-06), np.float64(3.1622776601683795e-05)),
 (np.float64(1e-06), np.float64(0.001)),
 (np.float64(1e-06), np.float64(0.03162277660168379)),
 (np.float64(1e-06), np.float64(1.0)),
 (np.float64(3.1622776601683795e-05), np.float64(1e-06)),
 (np.float64(3.1622776601683795e-05), np.float64(3.1622776601683795e-05)),
 (np.float64(3.1622776601683795e-05), np.float64(0.001)),
 (np.float64(3.1622776601683795e-05), np.float64(0.03162277660168379)),
 (np.float64(3.1622776601683795e-05), np.float64(1.0)),
 (np.float64(0.001), np.float64(1e-06)),
 (np.float64(0.001), np.float64(3.1622776601683795e-05)),
 (np.float64(0.001), np.float64(0.001)),
 (np.float64(0.001), np.float64(0.03162277660168379)),
 (np.float64(0.001), np.float64(1.0)),
 (np.float64(0.03162277660168379), np.float64(1e-06)),
 (np.float64(0.03162277660168379), np.float64(3.1622776601683795e-05)),
 (np.float64(0.03162277660168379), np.float64(0.001)),
 (np.float64(0.0316227766016837

In [55]:
base

[]

# Plot 6: Histogram showing differences in exchange fluxes when using extreme homeostatic and secretion weights